# **Análisis Exploratio de Datos**

#### **Conociendo mejor el conjunto de datos y respondiendo preguntas.**

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

----

**Cargar el conjunto de datos**

In [3]:
df_vehicle_price = pd.read_csv('../data/processed/vehicle_price.csv')
df_vehicle_price.head(5)

,Unnamed: 0,make,model,year,mileage,engine_hp,transmission,fuel_type,drivetrain,body_type,...,interior_color,owner_count,accident_history,seller_type,condition,trim,vehicle_age,mileage_per_year,brand_popularity,price
0,0,Volkswagen,Jetta,2016,183903,173,Manual,Electric,RWD,Sedan,...,Brown,5,No Accident,Dealer,Excellent,EX,9,20433.666667,0.040054,7208.52
1,1,Lexus,RX,2010,236643,352,Manual,Gasoline,FWD,Sedan,...,Beige,5,Minor,Dealer,Good,LX,15,15776.200000,0.039921,6911.81
2,2,Subaru,Crosstrek,2016,103199,188,Automatic,Diesel,AWD,Sedan,...,Beige,5,No Accident,Dealer,Excellent,Touring,9,11466.555556,0.040230,11915.63
3,3,Cadillac,Lyriq,2016,118889,338,Manual,Gasoline,AWD,SUV,...,Gray,3,No Accident,Private,Good,Base,9,13209.888889,0.039847,25984.79
4,4,Toyota,Highlander,2018,204170,196,Manual,Diesel,FWD,Sedan,...,Brown,5,Minor,Dealer,Excellent,Sport,7,29167.142857,0.039627,8151.30


**Identificar cuales columnas creemos que pueden tener un impacto en el valor del precio del vehiculo**

In [4]:
df_vehicle_price.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 21 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   Unnamed: 0        1000000 non-null  int64  
 1   make              1000000 non-null  object 
 2   model             1000000 non-null  object 
 3   year              1000000 non-null  int64  
 4   mileage           1000000 non-null  int64  
 5   engine_hp         1000000 non-null  int64  
 6   transmission      1000000 non-null  object 
 7   fuel_type         1000000 non-null  object 
 8   drivetrain        1000000 non-null  object 
 9   body_type         1000000 non-null  object 
 10  exterior_color    1000000 non-null  object 
 11  interior_color    1000000 non-null  object 
 12  owner_count       1000000 non-null  int64  
 13  accident_history  1000000 non-null  object 
 14  seller_type       1000000 non-null  object 
 15  condition         1000000 non-null  object 
 16  t

Las columnas que pueden llegar a influir en el precio del vehiculo son las siguientes:
- make o model
- year
- mileage
- fuel_type
- ower_count
- condition

Para confirmar nuestra intuisión realizaremos una **matriz de correlación** para confirmar si es verdad o no.